## 1. Setting up the Environment

In [ ]:
from dotenv import load_dotenv
from langchain_cerebras import ChatCerebras
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
import os

load_dotenv()
print("Environment and imports ready!")

### Essential imports for LangChain applications
- `dotenv`: Load environment variables from .env file  
- `ChatCerebras`: The LLM model we'll use
- `ChatPromptTemplate`: For creating structured prompts
- `tool`: Decorator for creating custom tools
- `create_react_agent`: For building AI agents that can use tools

## 2. Setting Up the ChatCerebras Model

LangChain supports various LLM providers. Here we'll use ChatCerebras, which requires an API key stored in your `.env` file as `CEREBRAS_API_KEY`.

In [ ]:
model_name = "qwen-3-32b"
api_key = os.getenv("CEREBRAS_API_KEY")

llm = ChatCerebras(
    model=model_name,
    api_key=api_key,
    temperature=0,
)

print(f"ChatCerebras model '{model_name}' initialized!")
print(f"API key loaded: {'Yes' if api_key else 'Missing CEREBRAS_API_KEY'}")

### Initialize the ChatCerebras model
- `temperature=0`: Makes responses deterministic (same input = same output)
- API key must be stored in `.env` file as `CEREBRAS_API_KEY`

## 3. Basic Chat with Prompt Templates

Let's start with a simple chat interaction using `ChatPromptTemplate`. This is perfect for straightforward question-answering scenarios.

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Provide clear, concise answers."),
    ("human", "{prompt}"),
])

# Example of few-shot prompting
template2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Provide clear, concise answers."),
    ("human", "What's the weather like today in France?"),
    ("assistant", "The weather in France today is sunny with a high of 25°C."),
    ("human", "{prompt}"),
])

llm_chain = template | llm

user_question = "What is the difference between a list and a tuple in Python?"
response = llm_chain.invoke({"prompt": user_question})

print("AI Response:")
print(response.content)

### Create a basic chat chain
- Template defines the conversation structure (system + human messages)
- Chain combines template with the LLM using the `|` operator
- `invoke()` runs the chain with input variables

## 4. Working with External Prompt Files

In real projects, it's better to store prompts in external files. Let's demonstrate how to load prompts from files.

In [ ]:
# In real projects, you would load system prompts from files

system_prompt = """You are an expert software engineer.
Your job is to help analyze code and provide technical assistance.
Always be precise and helpful in your responses.
"""
        
instructions = "Please analyze the following code for potential improvements."

### Loading prompts from external files
- Better practice: store prompts in separate `.md` files for easier editing
- Both solution files use this pattern for maintainable prompts

## 5. Creating Custom Tools

Tools are the building blocks of AI agents. They allow the AI to interact with the external world. Let's create some custom tools using the `@tool` decorator.

In [ ]:
@tool
def read_file_tool(file_path: str) -> str:
    """
    Read the contents of a file.

    Args:
        file_path (str): The path to the file to read.

    Returns:
        str: The contents of the file, or an error message if the file could not be read.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

@tool
def write_file_tool(file_path: str, content: str) -> str:
    """
    Write content to a file.

    Args:
        file_path (str): The path to the file to write.
        content (str): The content to write to the file.

    Returns:
        str: A message indicating success or failure.
    """
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing file: {str(e)}"

@tool
def create_directory_tool(directory_path: str) -> str:
    """
    Create a new directory.

    Args:
        directory_path (str): The path to the directory to create.

    Returns:
        str: A message indicating success or failure.
    """
    try:
        os.makedirs(directory_path, exist_ok=True)
        return f"Directory '{directory_path}' created successfully."
    except Exception as e:
        return f"Error creating directory: {str(e)}"

ALL_TOOLS = [read_file_tool, write_file_tool, create_directory_tool]

### Creating custom tools with @tool decorator
- Tools allow AI agents to interact with the external world
- Each tool needs a docstring that describes what it does
- The `ALL_TOOLS` list is used by the agent to access all available tools

## 6. Building AI Agents with LangGraph

Now for the exciting part! LangGraph allows us to create AI agents that can use tools to solve complex tasks. We'll use `create_react_agent` to build a ReAct (Reasoning and Acting) agent.

In [ ]:
agent_system_prompt = """
## Role
You are an expert software engineer.
Your job is to help analyze code and provide technical assistance.

## Style
Always be precise and helpful in your responses.

## Constraints
- Use the provided tools to read/write files and create directories as needed.
"""


agent = create_react_agent(
    model=llm,
    tools=ALL_TOOLS,
    prompt=system_prompt
)


print("AI Agent created successfully!")
print(f"Agent has access to {len(ALL_TOOLS)} tools")
print("Agent is ready to receive instructions")

### Building AI agents with LangGraph
- `create_react_agent` builds a ReAct (Reasoning and Acting) agent
- Agent can use tools to complete complex tasks step by step
- System prompt guides the agent's behavior